# Modeling

## Dependencies

In [1]:
from packages.helpers.helpers import joel_boto

## Functions

## Custom Variables

In [2]:
# Connect to custom AWS class
jb = joel_boto()

running local credentials
✅ Logged in to ECR successfully.
✅ Connected to all clients successfully.
